In [1]:
!pip install googlemaps

In [2]:
my_key="Add your googlemaps API key here"

In [3]:
import googlemaps
import pandas as pd
import time
import os

API_KEY = my_key
gmaps = googlemaps.Client(key=API_KEY)

keywords = [
    "hotels in Bahawalpur",
    "cheap hotels in Bahawalpur",
    "luxury hotels inBahawalpur",
    "lodges in Bahawalpur",
    "resorts in Bahawalpur",
    "guest houses in Bahawalpur",
]

csv_file = "hotels_reviews.csv"
data = []

def get_hotels_by_query(query):
    results = []
    next_page_token = None
    while True:
        response = gmaps.places(query=query, type="lodging", page_token=next_page_token)
        results.extend(response.get("results", []))
        next_page_token = response.get("next_page_token")
        if not next_page_token:
            break
        time.sleep(2)
    return results

def get_reviews_for_place(place_id):
    details = gmaps.place(place_id=place_id, fields=[
        'name', 'formatted_address', 'rating', 'price_level', 'url',
        'user_ratings_total', 'international_phone_number', 'reviews'
    ])
    return details.get('result', {})

for query in keywords:
    print(f"Searching: {query}")
    hotels = get_hotels_by_query(query)
    for hotel in hotels:
        hotel_info = get_reviews_for_place(hotel['place_id'])
        name = hotel_info.get('name', 'N/A')
        address = hotel_info.get('formatted_address', 'N/A')
        rating = hotel_info.get('rating', 'N/A')
        price_level = hotel_info.get('price_level', 'Not Provided')
        link = hotel_info.get('url', 'Not Available')
        phone = hotel_info.get('international_phone_number', 'Not Provided')
        total_reviews = hotel_info.get('user_ratings_total', 'N/A')
        reviews = hotel_info.get('reviews', [])

        for review in reviews:
            data.append({
                'City': "Bahawalpur",
                'Hotel Name': name,
                'Address': address,
                'Rating': rating,
                'Price Level': price_level,
                'Total Reviews': total_reviews,
                'Phone Number': phone,
                'Reviewer': review.get('author_name'),
                'Review Text': review.get('text'),
                'Review Rating': review.get('rating'),
                'Google Maps Link': link
            })

df = pd.DataFrame(data)

if os.path.exists(csv_file):
    df.to_csv(csv_file, mode='a', header=False, index=False)
else:
    df.to_csv(csv_file, index=False)

print(f"Saved {len(data)} reviews to {csv_file}")


Searching: hotels in Bahawalpur
Searching: cheap hotels in Bahawalpur
Searching: luxury hotels inBahawalpur
Searching: lodges in Bahawalpur
Searching: resorts in Bahawalpur
Searching: guest houses in Bahawalpur
Saved 542 reviews to hotels_reviews.csv


In [4]:
data=pd.read_csv("hotels_reviews.csv")
data[data["City"]=="Skardu"].count()

City                1517
Hotel Name          1517
Address             1517
Rating              1517
Price Level         1517
Total Reviews       1517
Phone Number        1517
Reviewer            1517
Review Text         1488
Review Rating       1517
Google Maps Link    1517
dtype: int64

In [5]:
data["City"].unique()

array(['Lahore', 'Islamabad', 'Rawalpindi', 'Karachi', 'Quetta', 'Multan',
       'Skardu', 'Muzaffarabad', 'Murree', 'Naran', 'Kaghan', 'Hyderabad',
       'Hunza', 'Swat', 'Bahawalpur'], dtype=object)